In [21]:

from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.layers import Layer
from keras import backend as K
import time
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
 
import fr_utils
from inception_blocks_v2 import *

import sys
np.set_printoptions(threshold=sys.maxsize) 

FRmodel = faceRecoModel(input_shape=(96, 96, 3))


In [22]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [40]:


def triplet_loss(y_true, y_pred, alpha = 0.2):

    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    #第一步：计算"anchor" 与 "positive"之间编码的距离，这里需要使用axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    
    
    #第二步：计算"anchor" 与 "negative"之间编码的距离，这里需要使用axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    
    #第三步：减去之前的两个距离，然后加上alpha
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    
    
    #通过取带零的最大值和对训练样本的求和来计算整个公式
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))
    
    return loss


In [41]:
with tf.compat.v1.Session() as test:
    tf.compat.v1.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random.normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random.normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random.normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

loss = 528.1426


In [61]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
fr_utils.load_weights_from_FaceNet(FRmodel)

In [129]:
database = {}
from PIL import Image
for filename in os.listdir(r"./images"):
    img = Image.open("images/" + filename)
    img = img.resize((96,96),Image.ANTIALIAS)
    img.save("images/"+filename)
database["0002"]=fr_utils.img_to_encoding("images/00002_940422_fa.png", FRmodel)
database["0003"]=fr_utils.img_to_encoding("images/00003_931230_fa_a.png", FRmodel)

In [132]:
# GRADED FUNCTION: who_is_it

def who_is_it(image_path, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    encoding = fr_utils.img_to_encoding(image_path, model)
    
 
    min_dist = 100

    for name, db_enc in database.items():

        dist = np.linalg.norm(db_enc - encoding)

        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print (image_path + "it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [133]:
for filename in os.listdir(r"./images"):
    who_is_it("images/"+filename, database, FRmodel)

images/00003_931230_fb_a.pngit's 0003, the distance is 0.5148776
images/00002_940928_fb.pngit's 0003, the distance is 0.6417244
images/00003_931230_fa_a.pngit's 0003, the distance is 0.0
images/00002_940928_rc.pngit's 0002, the distance is 0.60909384
images/00002_940422_fa.pngit's 0002, the distance is 0.0
Not in the database.
images/00003_930831_fb_a.pngit's 0002, the distance is 0.6862877
images/00002_940128_fb.pngit's 0002, the distance is 0.39083803
images/00002_930831_fa.pngit's 0002, the distance is 0.6686023
images/00002_940128_fa.pngit's 0002, the distance is 0.5698735
images/00002_930831_fb.pngit's 0002, the distance is 0.67760354
